In [18]:
#r "nuget: QuanTAlib"
#r "nuget: Plotly.NET;"
#r "nuget: Plotly.NET.Interactive;"

using QuanTAlib;
using Plotly.NET;
using Plotly.NET.LayoutObjects;

Installed Packages QuanTAlib, 0.2.0

In [19]:
string symbol="SPY"; // we'll focus on SPY symbol
int warmup = 50; // we'll allow 50 bars to pass by with no trading - for warm-up
Yahoo_Feed bars = new(symbol,350); //collect bars of symbol from Yahoo feed
TSeries data = bars.Close; //we need just one average value - (Open+High+Low+CLose)/4

//make a chart
var d = Chart2D.Chart.Candlestick<double, double, double, double, DateTime, string>(bars.Open.v.Skip(warmup).ToList(), bars.High.v.Skip(warmup).ToList(), 
bars.Low.v.Skip(warmup).ToList(), bars.Close.v.Skip(warmup).ToList(), bars.Open.t.Skip(warmup).ToList(), symbol)
    .WithSize(1200,400).WithMargin(Margin.init<int, int, int, int, int, bool>(30,10,40,30,1,false)).WithXAxisRangeSlider(RangeSlider.init(Visible:false)).WithTitle(symbol);
d

<!-- Plotly chart will be drawn inside this DIV -->

In [20]:
JMA_Series fastMA = new(data,12); //typically MACD uses EMA(12), but let's try with superior JMA(12)
JMA_Series slowMA = new(data,26); //likewise, let's use JMA(26) instead of EMA(26)

//make a chart
var cfast = Chart2D.Chart.Line<DateTime,double,bool>(fastMA.t.Skip(warmup).ToList(), fastMA.v.Skip(warmup).ToList(), false, "Fast MA").WithLineStyle(Width: 2, Color: Color.fromString("blue"));
var cslow = Chart2D.Chart.Line<DateTime,double,bool>(slowMA.t.Skip(warmup).ToList(), slowMA.v.Skip(warmup).ToList(), false, "Slow MA").WithLineStyle(Width: 2, Color: Color.fromString("red"));
var chart = Chart.Combine(new []{cfast,cslow}).WithSize(1200,400).WithMargin(Margin.init<int, int, int, int, int, bool>(30,10,40,30,1,false))
            .WithXAxisRangeSlider(RangeSlider.init(Visible:false)).WithTitle($"slow MA and fast MA of {symbol} OHLC4");
chart

<!-- Plotly chart will be drawn inside this DIV -->

In [21]:
SUB_Series MACD_line = new(fastMA, slowMA); //MACD line is just a SUBtraction fastMA-slowMA
JMA_Series signal_line = new(MACD_line, 9); //signal line is an EMA(9) of MACD; we use superior JMA(9) instead

//make a chart
var cfast = Chart2D.Chart.Line<DateTime,double,bool>(MACD_line.t.Skip(warmup).ToList(), MACD_line.v.Skip(warmup).ToList(), false, "MACD").WithLineStyle(Width: 2, Color: Color.fromString("orange"));
var cslow = Chart2D.Chart.Line<DateTime,double,bool>(signal_line.t.Skip(warmup).ToList(), signal_line.v.Skip(warmup).ToList(), false, "signal").WithLineStyle(Width: 2, Color: Color.fromString("green"));
var chart = Chart.Combine(new []{cfast,cslow}).WithSize(1200,400).WithMargin(Margin.init<int, int, int, int, int, bool>(30,10,40,30,1,false))
            .WithXAxisRangeSlider(RangeSlider.init(Visible:false)).WithTitle($"MACD line and signal line of fastMA-slowMA");
chart

<!-- Plotly chart will be drawn inside this DIV -->

In [22]:
SUB_Series histogram = new(MACD_line, signal_line); //MACD histogram is an oscillator of MACD-signal

//make a chart
var cfast = Chart2D.Chart.Column<double, DateTime, bool,bool,bool>(values: histogram.v.Skip(warmup).ToList(), Keys: histogram.t.Skip(warmup).ToList())
.WithSize(1200,400).WithMargin(Margin.init<int, int, int, int, int, bool>(30,10,40,30,1,false)).WithXAxisRangeSlider(RangeSlider.init(Visible:false))
.WithTitle("MACD histogram of MACD-signal");
cfast

<!-- Plotly chart will be drawn inside this DIV -->

In [23]:
COMPARE_Series over = new(histogram,0); //generate over/under series when histogram is above/below zero
CROSS_Series trades = new(histogram,0); //generate a signal series where histogram crosses zero (from below and from above)

//make a chart
var cover = Chart2D.Chart.Line<DateTime,double,bool>(over.t.Skip(warmup).ToList(),over.v.Skip(warmup).ToList(),false,"state").WithLineStyle(Width: 1, Color: Color.fromString("blue"));
var cbars = Chart2D.Chart.Area<DateTime, double,bool>(trades.t.Skip(warmup).ToList(), trades.v.Skip(warmup).ToList(),false );
var chart = Chart.Combine(new []{cover,cbars}).WithSize(1200,400).WithMargin(Margin.init<int, int, int, int, int, bool>(30,10,40,30,1,false))
            .WithXAxisRangeSlider(RangeSlider.init(Visible:false)).WithTitle("in-market signal and trading orders based on MACD histogram");
chart

<!-- Plotly chart will be drawn inside this DIV -->

In [25]:
EQUITY_Series folio = new(trades, data, Long:true, Short:false, Warmup:warmup); //generate equity curve from trades and 

//make a chart
var cbars = Chart2D.Chart.Area<DateTime, double,bool>(folio.t.Skip(warmup).ToList(), folio.v.Skip(warmup).ToList(),false ).WithSize(1200,400).WithMargin(Margin.init<int, int, int, int, int, bool>(30,10,40,30,1,false))
            .WithXAxisRangeSlider(RangeSlider.init(Visible:false)).WithTitle($"Trading P&L (long only) for MACD-generated trades on {symbol}");
cbars

<!-- Plotly chart will be drawn inside this DIV -->